In [19]:
from utils.load_results import *
from utils.plot_helpers import *

import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
plt.style.use('default')
import torch
from utils.analysis_from_interaction import *
from language_analysis_local import TopographicSimilarityConceptLevel, encode_target_concepts_for_topsim
import os
if not os.path.exists('analysis'):
    os.makedirs('analysis')
#import plotly.express as px

In [20]:
datasets = ['(3,4)', '(3,8)', '(3,16)', '(4,4)', '(4,8)', '(5,4)']
n_values = [4, 8, 16, 4, 8, 4]
n_attributes = [3, 3, 3, 4, 4, 5]
n_epochs = 300
n_datasets = len(datasets)
paths = ['results/' + d + '_game_size_10_vsf_3' for d in datasets]

In [21]:
datasets = ['(3,4)']
n_values = [4,]
n_attributes = [3,]
n_epochs = 300
n_datasets = len(datasets)
paths = ['results/' + d + '_game_size_10_vsf_3' for d in datasets]

In [22]:
context_unaware = False # whether original or context_unaware simulations are evaluated

setting = 'standard'
granularity_list = ('coarse', 'mixed', 'fine')

### Determine vocab size

In [23]:
from collections import Counter

In [24]:
# go through all datasets
for i, d in enumerate(datasets):
    for g in granularity_list:
        # select first run
        if g != 'mixed':
            print(g+ ' context condition')
            path_to_run = paths[i] + '/' + str(setting) +'/' + 'granularity_'+g+ '/'+ str(0) + '/'
            path_to_interaction_train = (path_to_run + 'interactions/train/epoch_' + str(n_epochs) + '/interaction_gpu0')
            path_to_interaction_val = (path_to_run + 'interactions/validation/epoch_' + str(n_epochs) + '/interaction_gpu0')
            interaction = torch.load(path_to_interaction_train)
            print(path_to_interaction_train)
            
            messages = interaction.message.argmax(dim=-1)
            messages = [msg.tolist() for msg in messages]
            all_symbols = [symbol for message in messages for symbol in message]
            symbol_counts = Counter(all_symbols)
            print(symbol_counts)
            print(len(symbol_counts), '\n')

coarse context condition
results/(3,4)_game_size_10_vsf_3/standard/granularity_coarse/0/interactions/train/epoch_300/interaction_gpu0
Counter({0: 742, 14: 450, 9: 391, 4: 387, 6: 295, 3: 196, 15: 153, 1: 124, 2: 100, 7: 47, 8: 46, 12: 37})
12 

fine context condition
results/(3,4)_game_size_10_vsf_3/standard/granularity_fine/0/interactions/train/epoch_300/interaction_gpu0
Counter({0: 742, 14: 332, 5: 315, 4: 214, 12: 205, 1: 193, 15: 187, 8: 168, 10: 151, 13: 149, 3: 148, 2: 95, 11: 46, 7: 23})
14 



In [27]:
import sys
import os
from collections import Counter

# Check if the output file exists
if not os.path.exists('vocab_sizes.txt'):
    mode = 'w'  # If the file doesn't exist, open in write mode to create it
else:
    mode = 'a'  # If the file exists, open in append mode to add to it

# Open the file in the appropriate mode
with open('vocab_sizes.txt', mode) as f:
    # Redirect stdout to the file
    sys.stdout = f
    
    # go through all datasets
    for i, d in enumerate(datasets):
        print('dataset: '+d+'\n')
        for g in granularity_list:
            # select first run
            if g != 'mixed':
                print(g+ ' context condition')
                path_to_run = paths[i] + '/' + str(setting) +'/' + 'granularity_'+g+ '/'+ str(0) + '/'
                path_to_interaction_train = (path_to_run + 'interactions/train/epoch_' + str(n_epochs) + '/interaction_gpu0')
                path_to_interaction_val = (path_to_run + 'interactions/validation/epoch_' + str(n_epochs) + '/interaction_gpu0')
                interaction = torch.load(path_to_interaction_train)
                print(path_to_interaction_train)
                
                messages = interaction.message.argmax(dim=-1)
                messages = [msg.tolist() for msg in messages]
                all_symbols = [symbol for message in messages for symbol in message]
                symbol_counts = Counter(all_symbols)
                print(symbol_counts)
                print(len(symbol_counts), '\n')

# Restore the original stdout
sys.stdout = sys.__stdout__


### See which messages are produced for which concepts